# 📰 Analyse des actualités agricoles de l'Agence Ecofin

Ce notebook a pour objectif de scraper des articles liés à la régulation des exportations agricoles dans les pays d’Afrique de l’Ouest, extraits du site [agenceecofin.com](https://www.agenceecofin.com/actualites-agro), puis d'extraire les données pertinentes et les sauvegarder dans un fichier Excel.

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import spacy

In [ ]:
# Charger le modèle de langue français de spaCy
nlp = spacy.load("fr_core_news_sm")

In [ ]:
# Initialisation des variables
mois_francais = {
    'janvier': '01', 'février': '02', 'fevrier': '02', 'mars': '03',
    'avril': '04', 'mai': '05', 'juin': '06',
    'juillet': '07', 'août': '08', 'aout': '08', 'septembre': '09',
    'octobre': '10', 'novembre': '11', 'décembre': '12', 'decembre': '12'
}

countries = ["Bénin", "Burkina Faso", "Cap-Vert", "Côte d'Ivoire", "Gambie", "Ghana", "Guinée",
             "Guinée-Bissau", "Libéria", "Mali", "Niger", "Nigéria", "Sénégal", "Sierra Leone", "Togo"]

keywords = [
    "interdiction des exportations agricoles", "licences d'exportation",
    "embargo sur les exportations agricoles", "limitation des exportations agricole",
    "exportation produits vivriers", "régulation", "suspension des exportations agricoles",
    "gouvernement suspend ", "restriction des exportations agricoles",
    "interdiction des exportations", "blocage", "plafonnement", "mesures protectionnistes",
    "interdiction exportation arachides", "produits agricoles exportés", "céréales",
    "blé", "riz", "maïs", "acajou", "amande karité", "fruits et légumes", "produits vivriers",
    "produits de rente", "pénurie", "tubercules", "niébé", "sorgho", "soja"
]

produits = ["maïs", "riz", "blé", "niébé", "sorgho", "soja", "cajou", "karité", "arachide", "mil", "tubercules"]

In [ ]:
# Lemmatisation des mots-clés
keywords_lemmatized = []
for phrase in keywords:
    doc = nlp(phrase.lower())
    keywords_lemmatized.append(" ".join([token.lemma_ for token in doc]))

In [ ]:
# Fonction de normalisation de texte
def normalize(text):
    return text.lower().strip()

In [ ]:
# Scraping (test sur 1 page pour commencer)
base_url = 'https://www.agenceecofin.com/actualites-agro'
data = []

for page in range(0, 10, 10):
    url = f'{base_url}?limitstart={page}'
    html_text = requests.get(url, timeout=50).text
    soup = BeautifulSoup(html_text, 'lxml')

    articles = soup.find_all('div', class_='catItemHeader')
    for article in articles:
        title_tag = article.find('div', class_='catItemTitle')
        if title_tag:
            title = title_tag.text.strip()
            if (any(country.lower() in title.lower() for country in countries) and
                any(keyword.lower() in title.lower() for keyword in keywords)):

                countrie = next((country for country in countries if country.lower() in title.lower()), "Inconnu")

                more_info = article.a['href']
                url_article = f'https://www.agenceecofin.com{more_info}'
                html_text2 = requests.get(url_article).text
                soup2 = BeautifulSoup(html_text2, 'lxml')
                read = soup2.find_all('div', class_='itemIntroText')

                published_date = soup2.find('span', class_='itemDateCreated').text.strip()
                match = re.search(r'(\d{1,2})\s+(\w+)\s+(\d{4})', published_date.lower())
                if match:
                    mois_nom = match.group(2)
                    annee = match.group(3)
                    mois_num = mois_francais.get(mois_nom)
                    published_date = f'{mois_num}-{annee}' if mois_num else "Date inconnue"
                else:
                    published_date = "Date inconnue"

                texte_complet = " ".join([normalize(p.text.strip()) for p in read])
                texte_final = re.split(r"lire\s+aussi[:\s]*", texte_complet, flags=re.IGNORECASE)[0].strip()
                texte_final = re.sub(r"\bmillions?\b|\bmilliards?\b|\bmilliers?\b|\bmilieu?\b", "", texte_final, flags=re.IGNORECASE)
                texte_final = re.sub(r"\s{2,}", " ", texte_final).strip()

                produits_trouves = [produit for produit in produits if normalize(produit) in texte_final]
                produits_uniques = list(set(produits_trouves))

                data.append({
                    "pays": countrie,
                    "title": title,
                    "date": published_date,
                    "article": texte_final,
                    "produits": ", ".join(produits_uniques)
                })

                time.sleep(10)

In [ ]:
# Conversion en DataFrame et aperçu
df = pd.DataFrame(data)
df.head()

In [ ]:
# Export en Excel
df.to_excel("articles_ecofin.xlsx", index=False)